In [1]:
import xarray as xr
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor 
from sklearn.model_selection import RandomizedSearchCV
from eofs.xarray import Eof

import cartopy.crs as ccrs
import matplotlib.pyplot as plt

In [21]:
# Utilities for normalizing the emissions data
min_co2 = 0.
max_co2 = 2400
def normalize_co2(data):
    return data / max_co2

min_ch4 = 0.
max_ch4 = 0.6
def normalize_ch4(data):
    return data / max_ch4

In [7]:
# functions to compute various NRMSEs
def get_nrmse_spatial(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    truth = truth.sel(time=slice(2080,None))
    pred = pred[-21:]
    truth_total = np.abs(truth.weighted(weights).mean(['lat', 'lon']).data.mean())
    rmse_spatial = np.sqrt(((truth - pred).mean('time')**2).weighted(weights).mean(['lat','lon'])).data
    return rmse_spatial / truth_total 

def get_nrmse_global(truth, pred):
    weights = np.cos(np.deg2rad(truth.lat))
    truth = truth.sel(time=slice(2080,None))
    pred = pred[-21:]
    truth_total = np.abs(truth.weighted(weights).mean(['lat', 'lon']).data.mean())
    rmse_global = np.sqrt((((truth - pred).weighted(weights).mean(['lat', 'lon']))**2).data.mean())
    return rmse_global / truth_total 

def get_nrmse(truth, pred):
    return get_nrmse_spatial(truth, pred) + 5 * get_nrmse_global(truth, pred)

In [8]:
# computes t-test for differences
def ttest_rel_from_stats(diff_mean, diff_std, diff_num):
    z = diff_mean / np.sqrt(diff_std ** 2 / diff_num)
    p = distributions.t.sf(np.abs(z), diff_num - 1) * 2
    return z, p

In [9]:
# compute vpd, given relative humidity and temperature
def get_vpd(humidity_data, tas_data):
    svp = 0.6112 * np.exp(17.76*(tas_data-273)/((tas_data-273) + 243.5))
    vpd = svp * (1 - humidity_data/100)
    return vpd

In [5]:
train_scenarios = ['historical', 'ssp126', 'ssp370', 'ssp585', 'hist-aer', 'hist-GHG']

In [16]:
Xtrain = xr.concat(
    [xr.open_dataset(f'train_val/inputs_{s}.nc') for s in train_scenarios]
    , dim='time')

In [18]:
# load and compute VPD for train and test data
hursTrain = xr.concat([xr.open_dataset(f'vpd_data/hurs_{s}.nc') for s in train_scenarios]
    , dim='time')
tasTrain = xr.concat([xr.open_dataset(f'vpd_data/tas_{s}.nc') for s in train_scenarios]
    , dim='time')
vpdTrain = get_vpd(hursTrain.hurs, tasTrain.tas).to_dataset(name='vpd')

hursTest = xr.open_dataset('vpd_data/hurs_ssp245.nc')
tasTest = xr.open_dataset('vpd_data/tas_ssp245.nc')
vpdTest = get_vpd(hursTest.hurs, tasTest.tas).to_dataset(name='vpd')

In [19]:
# add year data to X and Y
Xtrain["time"]=np.arange(1, 754)
vpdTrain["time"]=np.arange(1, 754)

In [22]:
# Create an EOF solver to do the EOF analysis. Square-root of cosine of
# latitude weights are applied before the computation of EOFs.
bc_solver = Eof(Xtrain['BC'])
so2_solver = Eof(Xtrain['SO2'])

In [23]:
# Retrieve the leading EOF, expressed as the correlation between the leading
# PC time series and the input SST anomalies at each grid point, and the
# leading PC time series itself.
bc_eofs = bc_solver.eofsAsCorrelation(neofs=5)
bc_pcs = bc_solver.pcs(npcs=5, pcscaling=1)

so2_eofs = so2_solver.eofsAsCorrelation(neofs=5)
so2_pcs = so2_solver.pcs(npcs=5, pcscaling=1)

In [24]:
# Convert the Principle Components of the aerosol emissions (calculated above) in to Pandas DataFrames
bc_df = bc_pcs.to_dataframe().unstack('mode')
bc_df.columns = [f"BC_{i}" for i in range(5)]

so2_df = so2_pcs.to_dataframe().unstack('mode')
so2_df.columns = [f"SO2_{i}" for i in range(5)]

In [25]:
# Bring the emissions data back together again and normalize
leading_historical_inputs = pd.DataFrame({
    "CO2": normalize_co2(Xtrain["CO2"].data),
    "CH4": normalize_ch4(Xtrain["CH4"].data)
}, index=Xtrain["CO2"].coords['time'].data)


In [26]:
# format Xtrain and ytrain to be used for random forest model
Xinput = pd.concat([leading_historical_inputs, bc_df, so2_df], axis=1).to_numpy()

In [27]:
Yinput = vpdTrain['vpd'].stack(dim=['lat','lon']).to_numpy()

In [28]:
# taken verbatim from Original_RF_model.ipynb
# parameters & hyperparameters for model

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5,55, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [5, 10, 15, 25]
# Minimum number of samples required at each leaf node
min_samples_leaf = [4, 8, 12]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# dictionary to hold all 4 random forest models
random_forest_regressor = RandomForestRegressor(random_state=0)

# create rscvs for each model
RSCV = RandomizedSearchCV(
        estimator = random_forest_regressor, 
        param_distributions = random_grid, 
        n_iter = 29, 
        cv = 3, 
        verbose=2, 
        n_jobs = -1
    )

# create random forests
random_forest = RSCV.fit(Xinput, Yinput)

Fitting 3 folds for each of 29 candidates, totalling 87 fits
